In [1]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

Using backend: pytorch


In [2]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [4]:
from dgl.data import citation_graph as citegrh
import networkx as nx

def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask

In [5]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
import time
import numpy as np

g, features, labels, train_mask, test_mask = load_cora_data()
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Extracting file to /home/claude/.dgl/cora


/home/claude/.virtualenvs/gnn/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/claude/.virtualenvs/gnn/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 1.9218 | Test Acc 0.3600 | Time(s) nan
Epoch 00001 | Loss 1.7895 | Test Acc 0.4140 | Time(s) nan
Epoch 00002 | Loss 1.6642 | Test Acc 0.4380 | Time(s) nan
Epoch 00003 | Loss 1.5616 | Test Acc 0.4750 | Time(s) 0.0482
Epoch 00004 | Loss 1.4745 | Test Acc 0.4890 | Time(s) 0.0515
Epoch 00005 | Loss 1.3896 | Test Acc 0.5050 | Time(s) 0.0495
Epoch 00006 | Loss 1.3057 | Test Acc 0.5150 | Time(s) 0.0514
Epoch 00007 | Loss 1.2258 | Test Acc 0.5130 | Time(s) 0.0497
Epoch 00008 | Loss 1.1510 | Test Acc 0.5220 | Time(s) 0.0491
Epoch 00009 | Loss 1.0819 | Test Acc 0.5430 | Time(s) 0.0503
Epoch 00010 | Loss 1.0161 | Test Acc 0.5680 | Time(s) 0.0509
Epoch 00011 | Loss 0.9525 | Test Acc 0.5970 | Time(s) 0.0526
Epoch 00012 | Loss 0.8912 | Test Acc 0.6250 | Time(s) 0.0524
Epoch 00013 | Loss 0.8330 | Test Acc 0.6410 | Time(s) 0.0535
Epoch 00014 | Loss 0.7782 | Test Acc 0.6670 | Time(s) 0.0545
Epoch 00015 | Loss 0.7268 | Test Acc 0.6860 | Time(s) 0.0546
Epoch 00016 | Loss 0.6792 | Test 